# 9 추천 시스템

## 9-1 추천 시스템의 개요와 배경

### 추천 시스템의 개요와 배경

### 온라인 스토어의 필수 요소, 추천 시스템

### 추천 시스템의 유형

## 9-2 콘텐츠 기반 필터링 추천 시스템

## 9-3 최근접 이웃 협업 필터링

## 9-4 잠재 요인 협업 필터링

### 잠재 요인 협업 필터링의 이해

### 행렬 분해의 이해

### 확률적 경사 하강법을 이용한 행렬 분해

In [ ]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재 요인 차원 K는 3으로 설정.
R = np.array([[4 , np.NaN , np.NaN , 2 , np.NaN ],
              [np.NaN, 5 , np.NaN , 3 , 1],
              [np.NaN , np.NaN , 3 , 4 , 4],
              [5,2,1,2,np.NaN]])
num_users , num_items = R.shape
K = 3

# P와 Q 행렬의 크기를 지정하고 정규분포를 가진 임의의 값으로 입력합니다.
np.random.seed(1)
P = np.random.normal(scale = 1./K , size = (num_users , K))
Q = np.random.normal(scale = 1./K , size = (num_items , K))

In [27]:
from sklearn.metrics import mean_squared_error

def get_rmse(R , P , Q, non_zeros):
    error = 0
    # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R행렬 생성
    full_pred_matrix = np.dot(P , Q.T)

    # 식제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind , y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind , y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros , full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [ ]:
# R > 0 인 행 위치 , 열 위치 , 값을 non_zeros 리스트에 저장.
non_zeros = [ (i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

steps = 1000
learning_rate = 0.01 
r_lambda = 0.01

# SGD 기법으로 P와 Q매트릭스를 계속 업데이트
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측값의 차이인 오류 값 구함
        eij = r - np.dot(P[i , : ] , Q[j , :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i , :] = P[i , :] + learning_rate*(eij * Q[j,:] - r_lambda*P[i,:])
        Q[j , :] = Q[j , :] + learning_rate*(eij * P[i,:] - r_lambda*Q[j,:])

    rmse = get_rmse(R,P,Q,non_zeros)
    if (step % 50) == 0:
        print('### iteration step : ' , step , " emse : " , rmse)

### iteration step :  0  emse :  3.2388050277987723
### iteration step :  50  emse :  0.4876723101369647
### iteration step :  100  emse :  0.15643403848192458
### iteration step :  150  emse :  0.07455141311978064
### iteration step :  200  emse :  0.043252267985793146
### iteration step :  250  emse :  0.029248328780879226
### iteration step :  300  emse :  0.022621116143829507
### iteration step :  350  emse :  0.019493636196525232
### iteration step :  400  emse :  0.018022719092132773
### iteration step :  450  emse :  0.01731968595344283
### iteration step :  500  emse :  0.016973657887570985
### iteration step :  550  emse :  0.01679680459589558
### iteration step :  600  emse :  0.016701322901884634
### iteration step :  650  emse :  0.016644736912476574
### iteration step :  700  emse :  0.016605910068210012
### iteration step :  750  emse :  0.01657420047570488
### iteration step :  800  emse :  0.01654431582921612
### iteration step :  850  emse :  0.016513751774735037
### i

In [ ]:
pred_matrix = np.dot(P , Q.T)
print('예측 행렬:\n', np.round(pred_matrix , 3))

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]


## 9-5 콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터 세트

### 장르 속성을 이용한 영화 콘텐츠 기반 필터링

### 데이터 로딩 및 가공

In [6]:
%cd /content/drive/MyDrive/AI/파이썬 머신러닝 완벽가이드/data_sets/archive (2)

/content/drive/MyDrive/AI/파이썬 머신러닝 완벽가이드/data_sets/archive (2)


In [7]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv('tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [8]:
movies_df = movies[['id' , 'title' , 'genres' , 'vote_average' , 'vote_count' , 'popularity' , 'keywords' , 'overview']]

In [9]:
pd.set_option('max_colwidth' , 100)
movies_df[['genres' , 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [10]:
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [11]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x ])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
movies_df[['genres' , 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


### 장르 콘텐츠 유사도 측정

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환.
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x) )
count_vect = CountVectorizer(min_df=0 , ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat , genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


In [16]:
genre_sim_sorted_ind = genre_sim.argsort()[ : , ::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


### 장르 콘텐츠 필터링을 이용한 영화 추천

In [17]:
def find_sim_movie(df , sorted_ind , title_name , top_n=10):

    # 인자로 입력된 movies_df DataFrame에서 'title' 칼럼이 입력된 title_name 값인 DataFrame 추출
    title_movie = df[df['title'] == title_name]

    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index , :(top_n)]
    
    # 추출돤 top_n index 출력. top_n index는 2차원 데이터임.
    # dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

In [18]:
similar_movies = find_sim_movie(movies_df , genre_sim_sorted_ind, 'The Godfather' , 10)
similar_movies[['title' , 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [20]:
movies_df[['title' , 'vote_average' , 'vote_count']].sort_values('vote_average' , ascending = False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


In [26]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C:' , round(C,3) , 'm:' , round(m,3))

C: 6.092 m: 370.2


In [27]:
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']

    return ( (v/(v+m) * R) + ( (m/(m+v)) * C) )

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average , axis = 1)

In [28]:
movies_df[['title' , 'vote_average' , 'weighted_vote' , 'vote_count']].sort_values('weighted_vote' , ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [29]:
def find_sim_movie(df , sorted_ind , title_name , top_n = 10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values

    # top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
    similar_indexes = sorted_ind[title_index , :(top_n * 2)]
    similar_indexes = similar_indexes.reshape(-1)
    # 기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]

    # top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote' , ascending = False)[:top_n]

similar_movies = find_sim_movie(movies_df , genre_sim_sorted_ind , 'The Godfather' , 10)
similar_movies[['title' , 'vote_average' , 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


## 9-6 아이템 기반 최근접 이웃 협업 필터링 실습

In [1]:
%cd /content/drive/MyDrive/AI/파이썬 머신러닝 완벽가이드/data_sets/movielens

/content/drive/MyDrive/AI/파이썬 머신러닝 완벽가이드/data_sets/movielens


### 데이터 가공 및 변환

In [3]:
import pandas as pd
import numpy as np

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
print(movies.shape)
print(ratings.shape)

(9125, 3)
(100004, 4)


In [4]:
ratings = ratings[['userId' , 'movieId' , 'rating']]
ratings_matrix = ratings.pivot_table('rating' , index = 'userId' , columns='movieId')
ratings_matrix.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,...,155820,156025,156387,156607,156609,156726,157200,157296,157407,157667,158238,158314,158528,158956,159093,159462,159690,159755,159858,159972,160080,160271,160438,160440,160563,160565,160567,160590,160656,160718,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# title 칼럼을 얻기 위해 movies와 조인
rating_movies = pd.merge(ratings , movies , on = 'movieId')

#columns = 'title'로 title 칼럼으로 피벗 수행.
ratings_matrix = rating_movies.pivot_table('rating' , index = 'userId' , columns='title') 

#NaN 값을 모두 0으로 변환
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(3) 

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And God Spoke (1993),...And Justice for All (1979),1-900 (06) (1994),10 (1979),10 Attitudes (2001),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Rifles (1969),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),"10th Kingdom, The (2000)","11'09""01 - September 11 (2002)",11:14 (2003),12 Angry Men (1957),12 Angry Men (1997),12 Years a Slave (2013),127 Hours (2010),12:08 East of Bucharest (A fost sau n-a fost?) (2006),13 Assassins (Jûsan-nin no shikaku) (2010),13 Ghosts (1960),13 Going on 30 (2004),13 Tzameti (2005),"13th Warrior, The (1999)",...,Zach Galifianakis: Live at the Purple Onion (2006),Zack and Miri Make a Porno (2008),Zardoz (1974),Zathura (2005),Zatoichi on the Road (Zatôichi kenka-tabi) (Zatôichi 5) (1963),Zazie dans le métro (1960),"Zed & Two Noughts, A (1985)",Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zenon: Girl of the 21st Century (1999),Zenon: The Zequel (2001),Zenon: Z3 (2004),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zerophilia (2005),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Holocaust (a.k.a. Doctor Butcher M.D.) (Zombi Holocaust) (1980),Zombieland (2009),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoot Suit (1981),Zootopia (2016),Zorba the Greek (Alexis Zorbas) (1964),Zorn's Lemma (1970),"Zorro, the Gay Blade (1981)",Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 영화 간 유사도 산출 

In [7]:
ratings_matrix_T =  ratings_matrix.transpose()
ratings_matrix_T.head(3)

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
$9.99 (2008),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(ratings_matrix_T , ratings_matrix_T)

# cosine_similarity()로 반환된 넘파이 행렬을 영화명을 매칭해 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim , index=ratings_matrix.columns , columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(3)

(9064, 9064)


title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And God Spoke (1993),...And Justice for All (1979),1-900 (06) (1994),10 (1979),10 Attitudes (2001),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Rifles (1969),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),"10th Kingdom, The (2000)","11'09""01 - September 11 (2002)",11:14 (2003),12 Angry Men (1957),12 Angry Men (1997),12 Years a Slave (2013),127 Hours (2010),12:08 East of Bucharest (A fost sau n-a fost?) (2006),13 Assassins (Jûsan-nin no shikaku) (2010),13 Ghosts (1960),13 Going on 30 (2004),13 Tzameti (2005),"13th Warrior, The (1999)",...,Zach Galifianakis: Live at the Purple Onion (2006),Zack and Miri Make a Porno (2008),Zardoz (1974),Zathura (2005),Zatoichi on the Road (Zatôichi kenka-tabi) (Zatôichi 5) (1963),Zazie dans le métro (1960),"Zed & Two Noughts, A (1985)",Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zenon: Girl of the 21st Century (1999),Zenon: The Zequel (2001),Zenon: Z3 (2004),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zerophilia (2005),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Holocaust (a.k.a. Doctor Butcher M.D.) (Zombi Holocaust) (1980),Zombieland (2009),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoot Suit (1981),Zootopia (2016),Zorba the Greek (Alexis Zorbas) (1964),Zorn's Lemma (1970),"Zorro, the Gay Blade (1981)",Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",1.0,0.0,0.0,0.164399,0.020391,0.0,0.014046,0.000000,0.0,0.003166,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.146780,0.0,0.017329,0.748926,0.164399,0.133791,0.069276,0.0,0.299751,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
$9.99 (2008),0.0,1.0,0.0,0.000000,0.000000,0.0,0.000000,0.079474,0.0,0.156330,0.108103,0.0,0.077954,0.0,0.0,0.0,0.0,0.0,0.026649,0.0,0.000000,0.000000,0.000000,0.028796,0.048655,0.0,0.000000,0.0,0.0,0.0,0.039008,0.0,0.0,0.061295,0.365636,0.0,0.0,0.000000,0.0,0.105823,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.145133,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071848,0.0,0.0,0.0,0.047953,0.031535,0.0,0.0,0.0,0.0,0.0,0.365636,0.0,0.0,0.0,0.0,0.0,0.0,0.013899,0.0,0.058218,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,1.0,0.000000,0.000000,1.0,0.000000,0.217357,0.0,0.000000,0.413919,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.109326,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.115642,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.218582,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0


In [9]:
item_sim_df["Godfather, The (1972)"].sort_values(ascending=False)[:6]

title
Godfather, The (1972)                        1.000000
Godfather: Part II, The (1974)               0.773685
Goodfellas (1990)                            0.620349
One Flew Over the Cuckoo's Nest (1975)       0.568244
American Beauty (1999)                       0.557997
Star Wars: Episode IV - A New Hope (1977)    0.546750
Name: Godfather, The (1972), dtype: float64

In [10]:
item_sim_df["Inception (2010)"].sort_values(ascending=False)[1:6]

title
Dark Knight, The (2008)          0.656999
Avatar (2009)                    0.633687
Dark Knight Rises, The (2012)    0.589308
Inglourious Basterds (2009)      0.580237
District 9 (2009)                0.576054
Name: Inception (2010), dtype: float64

### 아이템 기반 최근접 이웃협업 필터링으로 개인화된 영화 추천

In [11]:
def predict_rating(ratings_arr , item_sim_arr ):
    ratings_pred = ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1) ] )
    return ratings_pred

In [13]:
ratings_pred = predict_rating(ratings_matrix.values , item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred , index=ratings_matrix.index, columns = ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And God Spoke (1993),...And Justice for All (1979),1-900 (06) (1994),10 (1979),10 Attitudes (2001),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Rifles (1969),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),"10th Kingdom, The (2000)","11'09""01 - September 11 (2002)",11:14 (2003),12 Angry Men (1957),12 Angry Men (1997),12 Years a Slave (2013),127 Hours (2010),12:08 East of Bucharest (A fost sau n-a fost?) (2006),13 Assassins (Jûsan-nin no shikaku) (2010),13 Ghosts (1960),13 Going on 30 (2004),13 Tzameti (2005),"13th Warrior, The (1999)",...,Zach Galifianakis: Live at the Purple Onion (2006),Zack and Miri Make a Porno (2008),Zardoz (1974),Zathura (2005),Zatoichi on the Road (Zatôichi kenka-tabi) (Zatôichi 5) (1963),Zazie dans le métro (1960),"Zed & Two Noughts, A (1985)",Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zenon: Girl of the 21st Century (1999),Zenon: The Zequel (2001),Zenon: Z3 (2004),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zerophilia (2005),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Holocaust (a.k.a. Doctor Butcher M.D.) (Zombi Holocaust) (1980),Zombieland (2009),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoot Suit (1981),Zootopia (2016),Zorba the Greek (Alexis Zorbas) (1964),Zorn's Lemma (1970),"Zorro, the Gay Blade (1981)",Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.014679,0.012177,0.008457,0.155729,0.004635,0.008457,0.012224,0.011612,0.011952,0.007415,0.011565,0.006734,0.014505,0.018547,0.006750,0.000000,0.006777,0.011564,0.010426,0.005313,0.006429,0.001348,0.155729,0.013706,0.014109,0.024085,0.009659,0.000000,0.004278,0.003697,0.015351,0.010987,0.004123,0.006556,0.013833,0.007664,0.023125,0.006470,0.002161,0.014847,...,0.012863,0.008295,0.023060,0.006360,0.000000,0.007951,0.012701,0.006384,0.000000,0.007799,0.000000,0.000000,0.000000,0.006795,0.012713,0.004882,0.000000,0.007978,0.005313,0.009342,0.000000,0.007369,0.009627,0.011933,0.002309,0.000000,0.006591,0.018100,0.013833,0.026588,0.020867,0.005313,0.004119,0.012122,0.029437,0.009230,0.000000,0.015098,0.004838,0.000000
2,0.034046,0.032670,0.175621,0.031371,0.014654,0.175621,0.047991,0.057101,0.054119,0.033462,0.046144,0.089869,0.053359,0.068709,0.025026,0.023568,0.019461,0.023766,0.054699,0.017481,0.030011,0.064186,0.031371,0.068257,0.056756,0.122398,0.070888,0.012986,0.061805,0.083198,0.061828,0.127487,0.035218,0.028262,0.030081,0.022845,0.066959,0.055589,0.019362,0.066794,...,0.057855,0.026837,0.056547,0.036241,0.050500,0.009788,0.041326,0.056530,0.136400,0.023312,0.023568,0.023568,0.023568,0.021702,0.053177,0.018672,0.023568,0.027428,0.017481,0.027745,0.019153,0.029795,0.044795,0.069989,0.031467,0.121184,0.028521,0.041098,0.030081,0.038435,0.052653,0.017481,0.022326,0.042259,0.170733,0.045483,0.015784,0.062860,0.005560,0.013151
3,0.008073,0.031750,0.114354,0.035292,0.008948,0.114354,0.025436,0.039356,0.028688,0.044235,0.032779,0.062315,0.028017,0.058991,0.016194,0.016258,0.020503,0.040991,0.047909,0.017771,0.042582,0.069910,0.035292,0.043279,0.035759,0.068769,0.065558,0.022667,0.063509,0.074690,0.047315,0.104168,0.073859,0.036987,0.021281,0.028371,0.067417,0.045011,0.034048,0.057012,...,0.067239,0.033263,0.0655

In [14]:
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE를 구함.
def get_mse(pred, actual):
    # 평점이 있는 실제 영화만 추출
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred , actual)

print('아이템 기반 모든 최근접 이웃 MSE: ', get_mse(ratings_pred , ratings_matrix.values ))

아이템 기반 모든 최근접 이웃 MSE:  10.255398119852726


In [15]:
def predict_rating_topsim(ratings_arr , item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 열 크기만큼 루프 수행.
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 인덱스 반환
        top_n_items = [np.argsort(item_sim_arr[: , col])[:-n-1:-1]]
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row , col] = item_sim_arr[col, :] [ top_n_items].dot(ratings_arr[row , :][top_n_items].T)
            pred[row , col] /= np.sum(np.abs(item_sim_arr[col, :] [ top_n_items]))
    
    return pred

In [16]:
ratings_pred = predict_rating_topsim(ratings_matrix.values , item_sim_df.values , n=20)
print('아이템 기반 최근접 TOP-20 이웃 MSE: ' , get_mse(ratings_pred , ratings_matrix.values ))

# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns = ratings_matrix.columns)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if sys.path[0] == '':


아이템 기반 최근접 TOP-20 이웃 MSE:  3.9706198358584595


In [17]:
user_rating_id = ratings_matrix.loc[9,:]
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:10]

title
As Good as It Gets (1997)                      5.0
Fargo (1996)                                   5.0
William Shakespeare's Romeo + Juliet (1996)    5.0
Shadowlands (1993)                             5.0
Matrix, The (1999)                             5.0
Schindler's List (1993)                        5.0
Dead Man Walking (1995)                        5.0
Truman Show, The (1998)                        5.0
Edward Scissorhands (1990)                     4.0
Ever After: A Cinderella Story (1998)          4.0
Name: 9, dtype: float64

In [20]:
def get_unseen_movies(ratings_matrix , userId):
    # userId로 입력받은 사용자의 모든 영화 정보를 추출해 Series로 반환함
    # 반환된 user_rating은 영화명(title)을 인덱스로 가지는 Series 객체임.
    user_rating = ratings_matrix.loc[userId, :]

    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 인덱스를 추출해 list 객체로 만듦.
    already_seen = user_rating[ user_rating > 0 ].index.tolist()

    # 모든 영화명을 list객체로 만듦
    movies_list = ratings_matrix.columns.tolist()

    # list comprehension으로 already_seen에 해당하는 영화는 movies_list에서 제외함.
    unseen_list = [movie for movie in movies_list if movie not in already_seen ]

    return unseen_list

In [21]:
def recomm_movie_by_userid(pred_df, userId , unseen_list , top_n = 10):
    # 예측 평점 DataFram에서 사용자 id 인덱스와 unseen_list로 들어온 영화명 칼럼을 추출해
    # 가장 예측 평점이 높은 순으로 정렬함.
    recomm_movies = pred_df.loc[userId , unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

# 사용자가 관람하지 않는 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 아이템 기반의 최근접 이웃 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix , 9 ,unseen_list , top_n = 10)

# 평점 데이터를 DataFrame으로 생성,
recomm_movies = pd.DataFrame(data = recomm_movies.values , index=recomm_movies.index , columns=['pred_score'])

recomm_movies

,pred_score
title,
American Beauty (1999),1.258391
Pulp Fiction (1994),1.075948
"Usual Suspects, The (1995)",1.063974
Jerry Maguire (1996),1.047494
"English Patient, The (1996)",1.028489
Forrest Gump (1994),1.021220
"Piano, The (1993)",1.008307
L.A. Confidential (1997),0.992033
Rain Man (1988),0.976421


## 9-7 행렬 분해를 이용한 잠재 요인 협업 필터링 실습

In [22]:
def matrix_factorization(R , K , steps = 200 , learning_rate = 0.01 , r_lambda = 0.01):
    num_users , num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규 분포를 가진 랜덤한 값을 입력합니다.
    np.random.seed(1)
    P = np.random.normal(scale=1./K , size= (num_users , K))
    Q = np.random.normal(scale=1./K , size= (num_items , K))

    prev_rmse = 10000
    break_count = 0

    # R > 0 인 행 위치 , 열 위치 , 값을 non_zeros 리스트 객체에 저장.
    non_zeros = [ (i , j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i , j] > 0]

    # SGC기법으로 P와 Q 매트릭스를 계속 업데이트
    for step in range(steps):
        for i, j , r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함.
            eij = r - np.dot(P[i,:] , Q[j,:].T )
            # Regularization을 반영한 SGD업데이트 공식 적용
            P[i,:] = P[i , :] + learning_rate * (eij * Q[j , :] - r_lambda*P[i,:])
            Q[j, :] = Q[j,:] +  learning_rate * (eij * P[i , :] - r_lambda*Q[j,:])

        rmse = get_rmse(R,P,Q,non_zeros)
        if (step % 10) == 0:
            print("### iteration step : ", step , " rmse : ", rmse)

    return P, Q

In [24]:
import pandas as pd
import numpy as np

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
ratings = ratings[['userId' , 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating' , index='userId' , columns='movieId')

# title 칼럼을 얻기 위해 movies와 조인 수행.
rating_movies = pd.merge(ratings , movies , on = 'movieId')
# columns='title' 로 title 칼럼으로 pivot 수행.
ratings_matrix = rating_movies.pivot_table('rating' , index='userId' , columns='title')

In [28]:
P ,Q = matrix_factorization(ratings_matrix.values , K=50 , steps=100 , learning_rate =0.01, r_lambda = 0.01 )
pred_matrix = np.dot(P, Q.T)

### iteration step :  0  rmse :  2.9320565818473825
### iteration step :  10  rmse :  0.7499900285865972
### iteration step :  20  rmse :  0.5275349643538034
### iteration step :  30  rmse :  0.38354378256464045
### iteration step :  40  rmse :  0.3017115996487719
### iteration step :  50  rmse :  0.2547302025929975
### iteration step :  60  rmse :  0.22614838269743007
### iteration step :  70  rmse :  0.20743903539394296
### iteration step :  80  rmse :  0.19434445700230893
### iteration step :  90  rmse :  0.18468747401810467


In [29]:
ratings_pred_matrix = pd.DataFrame(data=pred_matrix , index = ratings_matrix.index , columns = ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And God Spoke (1993),...And Justice for All (1979),1-900 (06) (1994),10 (1979),10 Attitudes (2001),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Rifles (1969),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),"10th Kingdom, The (2000)","11'09""01 - September 11 (2002)",11:14 (2003),12 Angry Men (1957),12 Angry Men (1997),12 Years a Slave (2013),127 Hours (2010),12:08 East of Bucharest (A fost sau n-a fost?) (2006),13 Assassins (Jûsan-nin no shikaku) (2010),13 Ghosts (1960),13 Going on 30 (2004),13 Tzameti (2005),"13th Warrior, The (1999)",...,Zach Galifianakis: Live at the Purple Onion (2006),Zack and Miri Make a Porno (2008),Zardoz (1974),Zathura (2005),Zatoichi on the Road (Zatôichi kenka-tabi) (Zatôichi 5) (1963),Zazie dans le métro (1960),"Zed & Two Noughts, A (1985)",Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zenon: Girl of the 21st Century (1999),Zenon: The Zequel (2001),Zenon: Z3 (2004),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zerophilia (2005),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Holocaust (a.k.a. Doctor Butcher M.D.) (Zombi Holocaust) (1980),Zombieland (2009),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoot Suit (1981),Zootopia (2016),Zorba the Greek (Alexis Zorbas) (1964),Zorn's Lemma (1970),"Zorro, the Gay Blade (1981)",Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.745341,2.928824,1.150022,0.317918,1.286991,1.871280,1.846075,2.646222,2.902585,2.661586,2.195464,0.608522,2.779955,1.928736,1.545512,2.338563,2.941525,1.963991,2.656876,1.693866,1.691960,2.322863,0.253956,2.108598,2.292702,1.972134,1.727097,2.214280,1.276599,2.388034,2.948001,3.189461,2.559385,2.782414,1.576416,2.135739,2.552227,1.804815,2.757562,2.506897,...,2.847287,2.405110,2.012200,1.852091,2.898773,1.843986,2.504323,1.972297,2.766976,2.607408,1.828715,1.859487,1.921942,2.804495,2.795092,1.403710,2.384020,2.641500,0.706774,2.245736,0.282026,2.673242,3.506715,1.670689,1.108641,1.017347,2.775908,2.544950,1.994129,2.635129,2.466481,0.665347,2.536329,1.512936,2.512019,2.210895,0.533123,2.700853,2.361938,2.139596
2,2.162452,3.686570,1.542740,0.318964,1.740465,2.584770,2.545283,3.196354,3.670584,2.873576,3.224473,0.593172,3.628137,2.284465,1.780033,3.182354,4.160034,2.567087,2.817603,1.739367,1.971055,2.944146,0.364861,3.047398,2.820072,2.891571,1.786848,2.854636,1.481706,3.216226,4.887338,4.115034,3.886016,2.856978,2.323690,3.096030,2.846373,1.783163,3.289695,3.306920,...,3.811646,2.033234,2.350330,2.667462,3.964178,2.695785,3.496884,2.501433,3.450694,3.561695,2.498734,2.486135,2.500743,3.574336,3.287285,1.957012,3.096782,3.175365,0.728084,3.421150,0.387124,3.380659,4.256298,2.074804,1.609069,1.222717,3.821394,3.252336,3.013211,3.526337,3.374175,0.777538,2.977572,1.327954,3.101721,3.230171,0.632209,2.661355,2.884804,2.845496
3,2.250018,3.652691,1.543458,0.380940,1.689739,2.633695,2.563878,3.315305,4.030388,3.184506,3.011114,0.726711,4.062456,2.951770,2.557927,3.394686,3.995415,2.527493,2.396328,2.491147,1.687145,3.092542,0.354290,2.542716,2.960834,2.919580,2.099275,2.627264,1.540570,3.170159,4.510544,4.282788,3.509313,3.666319,2.696857,3.207990,3.150883,2.933097,3.529145,3.176418,...,3.754212,3.242999,2.9230

In [31]:
# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix , 9)

# 잠재 요인 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix , 9 , unseen_list , top_n = 10)

# 평점 데이터를 DataFrame으로 생성,
recomm_movies = pd.DataFrame(data=recomm_movies.values , index = recomm_movies.index)
columns=['pred_score']
recomm_movies

,0
title,
"Lives of Others, The (Das leben der Anderen) (2006)",5.342580
Bowling for Columbine (2002),5.189271
Boys Don't Cry (1999),5.084508
WALL·E (2008),5.059675
Run Lola Run (Lola rennt) (1998),5.047097
"Big Lebowski, The (1998)",5.046238
Finding Nemo (2003),5.045660
"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",5.009302
Dancer in the Dark (2000),4.960101


## 9-8 파이썬 추천 시스템 패키지 - Surprise

In [32]:
from surprise import SVD

ModuleNotFoundError: ignored